In [50]:
import pandas as pd
from glob import glob

In [51]:
!pip3 install xlrd

In [52]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [53]:
filepaths_order = glob('sources/order_new/order*.xlsx')
filepaths_order

[]

In [54]:
filepath = filepaths_order[0]
filepath

IndexError: list index out of range

In [ ]:
df_order = pd.read_excel(filepath)
df_order

In [ ]:
df_order.to_dict().items()

In [ ]:
from collections import defaultdict

In [ ]:
order = defaultdict(int)

In [ ]:
for key, value in df_order.to_dict().items():
    order[key] += value[0]
#     print(key, value) 

In [ ]:
order

In [ ]:
filepath = filepaths_order[1]
df_order = pd.read_excel(filepath)
df_order

In [ ]:
for key, value in df_order.to_dict().items():
    order[key] += value[0]

In [ ]:
order

In [ ]:
import os
import shutil

In [ ]:
os.listdir('sources/order_new/')

In [ ]:
os.listdir('sources/order_old/')

In [ ]:
filepath = filepaths_order[0]
shutil.move(filepath, 'sources/order_old/')

In [ ]:
def update_order(order, filepath):
    # 注文表読み込み
    df_order = pd.read_excel(filepath)
    for key, value in df_order.to_dict().items():
        order[key] += value[0]
    # ファイルの移動
    shutil.move(filepath, 'sources/order_old/')
    return order

In [ ]:
order = defaultdict(int)

for filepath in filepaths_order:
    order = update_order(order, filepath)

In [ ]:
order

In [ ]:
order

In [56]:
filepath_stock = 'sources/stock.xlsx'
df_stock = pd.read_excel(filepath_stock)

df_stock.tail(3)

,日付,曜日,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
12,2021-02-02,Tue,66,48,38,70,31,18,98
13,2021-02-03,Wed,66,32,34,16,24,16,75
14,2021-02-03,Wed,66,16,30,-38,17,14,52


In [ ]:
pd.DataFrame([[10, 4, 6]])

In [ ]:
pd.DataFrame([[5, 1, 10]])

In [ ]:
pd.DataFrame([[10, 4, 6]]) - pd.DataFrame([[5, 1, 10]])

In [57]:
stock = df_stock.iloc[-1:, 2:]
stock

,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
14,66,16,30,-38,17,14,52


In [58]:
order = pd.DataFrame(order, index=[0], columns=stock.columns)
order

,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
order = order.fillna(0)
order

,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
0,0,0,0,0,0,0,0


In [60]:
stock - order

,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
stock = stock.reset_index(drop=True)

In [62]:
updated = stock - order
updated

,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
0,66,16,30,-38,17,14,52


In [63]:
filepath_master = 'sources/master.xlsx'
df_master = pd.read_excel(filepath_master)
df_master

,Unnamed: 0,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
0,閾値,30,20,20,20,30,10,40
1,追加量,100,80,100,100,80,80,120


In [64]:
df_master.iloc[0 , 1: ]

日本酒       30
焼酎        20
赤ワイン      20
白ワイン      20
ウィスキー     30
ブランデー     10
缶チューハイ    40
Name: 0, dtype: object

In [65]:
threshold = df_master.iloc[:1 , 1: ]
threshold

,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
0,30,20,20,20,30,10,40


In [66]:
updated

,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
0,66,16,30,-38,17,14,52


In [69]:
shortage_columns = updated[updated < threshold].dropna(axis=1).columns
shortage_columns

Index(['焼酎', '白ワイン', 'ウィスキー'], dtype='object')

In [72]:
df_shortage = df_master.iloc[1:, 1:][shortage_columns]
df_shortage

,焼酎,白ワイン,ウィスキー
1,80,100,80


In [75]:
order_text = ''
for key, value in df_shortage.to_dict().items():
    order_text += f'{key}を{value[1]}本、'
order_text += '発注お願いします。'

In [76]:
order_text

'焼酎を80本、白ワインを100本、ウィスキーを80本、発注お願いします。'

In [79]:
import json
with open('sources/secret.json') as f:
    address_password = json.load(f)

In [81]:
import smtplib
from email.mime.text import MIMEText
from email.utils import formatdate

from_addr = address_password['ADDRESS']
password = address_password['PASSWORD']

subject = '発注依頼'
body = f'**さん、在庫数が足りなくなってしまったため、{order_text}発注してください。'
to_addr = 'silent_quiet_d@yahoo.co.jp'

# SMTPサーバに接続
smtpobj = smtplib.SMTP('smtp.gmail.com', 587)
smtpobj.starttls()
smtpobj.login(from_addr, password)

# メッセージ（ルール）の作成
msg = MIMEText(body)
msg['Subject'] = subject
msg['From'] = from_addr
msg['To'] = to_addr
msg['Date'] = formatdate()

# 作成したメールを送信
smtpobj.send_message(msg)
smtpobj.close()




In [93]:
from datetime import datetime

In [94]:
today = datetime.today()

In [95]:
print(today.strftime('%a'))

Thu


In [96]:
updated['日付'] = today
updated['曜日'] = today.strftime('%a')

In [97]:
updated

,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ,日付,曜日
0,66,16,30,-38,17,14,52,2021-05-20 15:37:03.161599,Thu


In [99]:
pd.concat([df_stock, updated], sort=False).reset_index(drop=True).to_excel(filepath_stock, index=False)

In [100]:
pd.read_excel(filepath_stock)

,日付,曜日,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
0,2021-01-15 00:00:00.000,Fri,100,80,51,63,150,60,95
1,2021-01-18 00:00:00.000,Mon,88,73,35,38,130,50,82
2,2021-01-19 00:00:00.000,Tue,73,66,105,15,126,37,75
3,2021-01-20 00:00:00.000,Wed,50,55,96,123,114,27,43
4,2021-01-21 00:00:00.000,Thu,39,44,75,96,106,13,19
5,2021-01-22 00:00:00.000,Fri,25,31,64,88,98,10,117
6,2021-01-25 00:00:00.000,Mon,125,106,37,77,83,84,97
7,2021-01-26 00:00:00.000,Tue,120,96,34,50,69,75,67
8,2021-01-27 00:00:00.000,Wed,112,85,10,35,54,53,49
9,2021-01-28 00:00:00.000,Thu,103,74,90,19,41,44,12
